In [1]:
# # coding: utf-8
import pandas as pd
import numpy as np

import time

import data_load as dl

In [23]:
dl.get_data("""
select *
from tl.events

order by insert_time desc


limit 15


"""
)

,user_id,user_name,event_name,parameters,insert_time
0,249792088,rosolimo,button_click,{Button_name: Посмотреть список лучших игроков},2023-02-26 21:40:12.515715
1,249792088,rosolimo,button_click,{Button_name: Главное меню},2023-02-26 21:40:11.055830
2,765176011,SomeoneCalledAnna,button_click,{Button_name: Посмотреть рейтинги книг},2023-02-26 21:29:56.270119
3,765176011,SomeoneCalledAnna,button_click,{Button_name: Главное меню},2023-02-26 21:29:53.650573
4,765176011,SomeoneCalledAnna,button_click,{Button_name: Посмотреть мою индивидуальную статистику},2023-02-26 21:29:49.475340
5,765176011,SomeoneCalledAnna,button_click,{Button_name: Пропустить},2023-02-26 21:29:47.650674
6,765176011,SomeoneCalledAnna,button_click,{Button_name: Продолжить},2023-02-26 21:29:44.449084
7,765176011,SomeoneCalledAnna,button_click,{Button_name: Пропустить},2023-02-26 21:29:42.974939
8,765176011,SomeoneCalledAnna,button_click,{Button_name: Продолжить},2023-02-26 21:29:40.232312
9,765176011,SomeoneCalledAnna,button_click,{Button_name: Пропустить},2023-02-26 21:29:38.723863


In [20]:
dl.get_data("""
select distinct user_name, user_id
from tl.events



"""
)

,user_name,user_id
0,noramornomore,435209913
1,denivladislav,435124276
2,vladchernichenko,299230584
3,igormazur,212619715
4,SomeoneCalledAnna,765176011
5,tayistrash,508625689
6,nmilutina,39629726
7,al_iesh,539046387
8,eugeniabooo,1033768312
9,None,685490258


In [30]:
dl.get_data("""
select *
from tl.events
where user_name = 'al_iesh'--'noramornomore'


"""
)

,user_id,user_name,event_name,parameters,insert_time
0,539046387,al_iesh,session_start,,2023-02-26 20:35:31.611324
1,539046387,al_iesh,button_click,{Button_name: Начать игру},2023-02-26 20:35:40.155926
2,539046387,al_iesh,button_click,{Button_name: Фантастика},2023-02-26 20:35:44.657602
3,539046387,al_iesh,button_click,{Button_name: Божественная комедия},2023-02-26 20:35:52.198085
4,539046387,al_iesh,button_click,{Button_name: Продолжить},2023-02-26 20:35:55.611646
5,539046387,al_iesh,button_click,{Button_name: Колобок},2023-02-26 20:36:01.018096
6,539046387,al_iesh,button_click,{Button_name: Главное меню},2023-02-26 20:36:08.664123
7,539046387,al_iesh,button_click,{Button_name: Посмотреть мою индивидуальную статистику},2023-02-26 20:36:10.321997
8,539046387,al_iesh,button_click,{Button_name: Главное меню},2023-02-26 20:36:12.626303
9,539046387,al_iesh,button_click,{Button_name: Начать игру},2023-02-26 20:36:16.571791


In [30]:
dl.get_data("""
select pg_relation_size('tl.rating_history') / 1024

"""
)

,?column?
0,88


In [24]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select 
                            o."user"
                            , round(((o.attemps * o.original_score) + (o.attemps * (50 - non_random_rate))) / 100,1) as score
                            , round(o.attemps,0) as attemps 
                            , round(o.original_score,0) as original_score
                            , non_random_rate
                    from original o
                    left outer join random_rate rr on
                        (rr.user_id = o.user_id)
                    order by 2 desc



                    """)

,user,score,attemps,original_score,non_random_rate
0,tsaregalina,22.7,130,5,38
1,rosolimo,21.1,116,9,41
2,None,6.9,48,5,41
3,vladchernichenko,6.7,29,9,36
4,denivladislav,4.1,30,6,42
5,None,3.5,30,5,43
6,eugeniabooo,2.3,21,4,43
7,noramornomore,0.5,3,8,41
8,nmilutina,0.1,6,0,49
9,al_iesh,0.0,1,0,50


In [27]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select *
                    from indiv_stat
                    where total > 2
                    and "user" = 'eugeniabooo'

                    order by original_score desc



                    """)

,item,user_id,user,selected,total,select_rate,global_select_rate,original_score
0,Герой нашего времени,1033768312,eugeniabooo,1,3,0.333333,0.461538,13
1,Война и мир,1033768312,eugeniabooo,2,3,0.666667,0.615385,5
2,Преступление и наказание,1033768312,eugeniabooo,5,5,1.000000,0.540541,0
3,Колобок,1033768312,eugeniabooo,0,4,0.000000,0.300000,0
4,Лабиринт отражений,1033768312,eugeniabooo,0,4,0.000000,0.214286,0
5,Мастер и Маргарита,1033768312,eugeniabooo,3,3,1.000000,0.925000,0


In [10]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    ),
                    global_stat as 
                    (
                        select item
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item
                    ),
                    personal_stat as 
                    (
                        select item, user_id, "user"
                                    , sum(is_selected) as selected
                                    , count(is_selected) as total
                                    , sum(is_selected)::float / count(is_selected) as select_rate
                        from items
                        group by item, user_id, "user"
                    ),
                    indiv_stat as 
                    (
                        select ps.*
                                , gs.select_rate as global_select_rate
                                , case 
                                    when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                                    else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
                                end as original_score
                        from personal_stat ps
                        left outer join global_stat gs on 
                            (ps.item=gs.item)
                    ),
                    original as 
                    (
                        select user_id, "user"
                                , avg(original_score) as original_score, sum(total) / 2 as attemps
                        from  indiv_stat
                        group by user_id, "user"
                    ),
                    random_rate as 
                    (
                        select user_id, "user"
                                , stddev_pop(select_rate)
                                ,  case 
                                    when stddev_pop(select_rate) <= 0.1 then 0
                                    when stddev_pop(select_rate) >= 0.9 then 100
                                    else round((100*(stddev_pop(select_rate)))::decimal,0)
                                end as non_random_rate
                        from personal_stat
                        group by user_id, "user"
                    )


                    select user_id, "user", item
                                , count(distinct id) as attempts
                                , round(avg(is_selected),2) as select_rate
                                , round(stddev_pop(is_selected),2) as std
                    from items
                    where 
                    "user" = 'eugeniabooo'

                    group by user_id, "user", item

                    order by 4 desc


                    """)

,user_id,user,item,attempts,select_rate,std
0,1033768312,eugeniabooo,Преступление и наказание,5,1.00,0.00
1,1033768312,eugeniabooo,Колобок,4,0.00,0.00
2,1033768312,eugeniabooo,Лабиринт отражений,4,0.00,0.00
3,1033768312,eugeniabooo,Война и мир,3,0.67,0.47
4,1033768312,eugeniabooo,Герой нашего времени,3,0.33,0.47
5,1033768312,eugeniabooo,Мастер и Маргарита,3,1.00,0.00
6,1033768312,eugeniabooo,Унесённые ветром,2,0.00,0.00
7,1033768312,eugeniabooo,Волкодав,2,1.00,0.00
8,1033768312,eugeniabooo,Джейн Эйр,2,0.50,0.50
9,1033768312,eugeniabooo,"Дядя Фёдор, пёс и кот",2,0.00,0.00


In [33]:
fresh_ratings_df = dl.get_data("""
        with 
        base as 
        (
            select item, rating, insert_time
            from tl.rating_history

            --union all 

            --select item_name as item, start_rating as rating, insert_time
            --from tl.items
        ),
        fresh_note as
        (
            select *, max(insert_time) over (partition by item) as fresh_time
            from base
        )

        select item, rating
        from fresh_note
        where fresh_time = insert_time

        order by rating desc
                                """)

In [34]:
fresh_ratings_df

,item,rating
0,Герой нашего времени,1725
1,Лабиринт отражений,1712
2,Отверженные,1694
3,Унесённые ветром,1692
4,Янки при дворе короля Артура,1678
5,Властелин Колец,1671
6,Голодные игры,1668
7,Айвенго,1664
8,Незнайка на Луне,1662
9,Преступление и наказание,1654


In [29]:
dl.get_data("""
                    with
                    random_rate_old as 
                    (
                        select 
                        user_id, avg(result) as random_value, count(distinct insert_time) as attempts
                        from tl.game_results
                        group by user_id
                    ), 
                    left_tbl as 
                    (
                        select battle1 as item, user_id, "user"
                            , case 
                                when result = 0 then 1
                                else 0
                            end as is_selected
                                --, row_number() as id
                        from tl.game_results
                    ),
                    right_tbl as 
                    (
                            select battle2 as item, user_id, "user"
                                        , case 
                                            when result = 1 then 1
                                            else 0
                                        end as is_selected
                        from tl.game_results
                    ),
                    pre_items as 
                    (
                        select * from left_tbl

                        union all

                        select * from right_tbl
                    ),
                    items as 
                    (
                        select *, row_number() over() as id
                        from pre_items
                    )

                    select item, count(distinct id)
                    from items
                    group by item
                    order by count(distinct id) desc

                    limit 10


                    """)

,item,count
0,Мастер и Маргарита,57
1,Незнайка на Луне,50
2,Преступление и наказание,46
3,Ромео и Джульета,41
4,Айвенго,38
5,Гарри Поттер и философский камень,32
6,Три повести о Малыше и Карлсоне,27
7,Вокруг света за 80 дней,26
8,Колобок,25
9,"Дядя Фёдор, пёс и кот",24
